In [1]:
import xarray as xr
import numpy as np
import hydromt
from os.path import join, basename, isdir
import glob
import matplotlib.pyplot as plt
from rasterio.transform import from_bounds
import geopandas as gpd

In [2]:
from hydromt import DataCatalog
data_libs = [
    r'../../1_data/2_forcing/forcing_data.yml',    
]
data_cat = DataCatalog(deltares_data=True, data_libs=data_libs)

In [7]:
# read SFINCS maps
mdir = r'../../3_models/CMF/03min'

# from params.txt:
shape = 45, 35  # nrow, ncol 6 min
shape = 90, 70  # nrow, ncol 3 min
bbox = [32, -21.5, 35.5, -17] # W, S, E, N

# read model maps
transform = from_bounds(*bbox, shape[1], shape[0])
nodata=-9999
da_lst = []
nlayers = {'nextxy': 2, 'lonlat': 2}
dtypes = {'nextxy': 'i4', 'basin': 'i4'}
for name in ['elevtn', 'nextxy', 'lonlat', 'uparea', 'rivhgt', 'rivwth_gwdlr', 'basin', 'rivdph_hc0.27_hp0.3']:
    shape0 = (nlayers[name], shape[0], shape[1]) if name in nlayers else shape
    dtype = dtypes.get(name, 'f4')
    da = hydromt.raster.RasterDataArray.from_numpy(
        data = np.fromfile(join(mdir, f'{name}.bin'), dtype).reshape((shape0)),
        transform=transform, 
        nodata=nodata,
    )
    da.name = name
    da_lst.append(da)
ds = xr.merge(da_lst)#.rename({'rivhgt': 'rivdph', 'rivwth_grwl': 'rivwth'})
ds.raster.set_crs(4326)
ds['idx'] = xr.DataArray(data=np.arange(ds.raster.size, dtype=int).reshape(ds.raster.shape), dims=ds.raster.dims)
cmf_mask = ds['uparea'] != ds['uparea'].raster.nodata
ds['uparea'] = (ds['uparea']/1e6).where(cmf_mask, ds['uparea'].raster.nodata) # km2
ds['pit'] = (ds['nextxy']==-9).all('dim0')
cols, rows = np.meshgrid(range(shape[1]), range(shape[0]))
ds['col']= xr.Variable(data=cols+1, dims=da.raster.dims)
ds['row']= xr.Variable(data=rows+1, dims=da.raster.dims)

# initiate flow dir object
flw = hydromt.flw.flwdir_from_da(ds['nextxy'], ftype='nextxy')

In [11]:
ds['uparea'].raster.set_nodata(nodata)
ds['uparea'].raster.to_raster(join(mdir, 'uparea.tif'))
ds['elevtn'].raster.to_raster(join(mdir, 'elevtn.tif'))
ds['rivdph_hc0.27_hp0.3'].raster.to_raster(join(mdir, 'rivdph_hc0.27_hp0.3.tif'))

In [ ]:
# get CaMa-flood outlet locations and convert to point geometry
xs, ys = ds['lonlat'].raster.mask_nodata().values
gdf_out = gpd.GeoDataFrame(
    geometry=gpd.points_from_xy(xs.ravel(),ys.ravel(), crs=4326),
    data = {name: ds[name].values.ravel() for name in ds.data_vars if ds[name].ndim == 2}
)
gdf_out = gdf_out[~gdf_out.geometry.is_empty].set_index('idx')  # drop invalid  points
gdf_out['upa10'] = np.log10(gdf_out['uparea'])
gdf_pits = gdf_out[gdf_out['pit']]
gdf_pits.columns

### prepare downstream boundary mapping

In [ ]:
import pygeos

def cmf_pit_mapping(gdf_gtsm, name, gdf_pits, mapdir=mdir, max_dist=10e3):
    pts = pygeos.points([g.coords[:][0] for g in gdf_pits.geometry])
    gtsm_idx = gdf_gtsm.sindex.nearest(pts)[1]
    gdf_pits['gtsm_idx'] = gtsm_idx + 1
    gdf_pits['gtsm_dst']  = gdf_gtsm.iloc[gtsm_idx].to_crs(32736).distance(gdf_pits.to_crs(32736), align=False).values
    gdf_pits = gdf_pits[gdf_pits['gtsm_dst']<max_dist]
    nlinks = gdf_pits.index.size
    print(nlinks)
    cmf_gtsm_ref = gdf_pits[['col', 'row', 'gtsm_idx']].fillna(0).values.astype(np.int32)
    with open(join(mapdir, f'{name}.txt'), 'w') as f:
        f.write(f'{nlinks:d}\n')
        np.savetxt(
            f, 
            cmf_gtsm_ref,
            fmt='%3.d',
        )
    return gdf_pits, cmf_gtsm_ref

In [ ]:
gdf_gtsm = data_cat.get_geodataset('gtsm_idai_twl').vector.to_gdf()
gdf_pits1, cmf_gtsm_ref = cmf_pit_mapping(gdf_gtsm, name='GTSM_EVENT', gdf_pits=gdf_pits, max_dist=20e3)

fig, ax = plt.subplots(1,1, figsize=(6,6))
gdf_pits.plot(markersize=(gdf_pits['upa10'].values)*6, color='g', ax=ax)
gdf_pits1.plot(markersize=(gdf_pits1['upa10'].values)*7, color='k', ax=ax)
gdf_gtsm.iloc[np.unique(gdf_pits1['gtsm_idx'].values-1)].plot(markersize=15, color='r', marker='^', ax=ax)

### prepare river width and depth maps

In [ ]:
# get lin 2019 river width and bankfull Q data

bbox1 = np.nanmin(xs), np.nanmin(ys), np.nanmax(xs), np.nanmax(ys)
print(bbox1)

gdf_rivdata = data_cat.get_geodataframe('rivers_lin2019_v1', bbox=bbox1, buffer=1000) # buffer [m]

In [ ]:
# merge lin river with data with CaMa-Flood data
from hydromt.gis_utils import nearest_merge
cols = ["rivwth", "qbankfull"]
gdf_out1 = nearest_merge(gdf_out, gdf_rivdata, columns=cols, max_dist=2e2)
for col in cols:
    data = np.full(ds.raster.shape, nodata, dtype=float)
    data.flat[gdf_out1.index.values] = gdf_out1[col].fillna(nodata).values
    # ds[col] = xr.DataArray(flw.fillnodata(data, nodata), dims=ds.raster.dims)
    ds[col] = xr.DataArray(data, dims=ds.raster.dims)
    ds[col].raster.set_nodata(nodata)

In [ ]:
# derive river width 
min_rivwth = 5
min_rivdph = 1

ds['qbankfull'] = np.maximum(ds['qbankfull'], 0).where(cmf_mask, nodata)
ds['rivwth'] = np.maximum(ds['rivwth'], min_rivwth).where(cmf_mask, nodata)
# ds['qbankfull'].values.astype(np.float32).tofile(join(mdir, f'qbankfull.bin'))
# ds['rivwth'].values.astype(np.float32).tofile(join(mdir, f'rivwth_lin.bin'))

In [ ]:
# derive river depth
hc, hp = 0.27, 0.30
for hc in [0.135, 0.27, 0.405]:
    ds['rivdph'] = np.maximum(hc * ds['qbankfull']**hp, min_rivdph).where(cmf_mask, nodata)
    ds['rivdph'].values.astype(np.float32).tofile(join(mdir, f'rivdph_hc{hc}_hp{hp}.bin'))
    ds['rivdph'].raster.set_nodata(nodata)
    # ds['rivdph'].raster.to_raster(join(mdir, 'rivbath', f'rivdph_hc{hc}_hp{hp}.tif'))

In [ ]:
# get CaMa-Flood flow directions and vectorize streams for visualization
feats = flw.vectorize(
    xs=xs, ys=ys, **{name: ds[name].values for name in ds.data_vars if (ds[name].ndim == 2 and name != 'idx')}
)
gdf_riv0 = gpd.GeoDataFrame.from_features(feats, crs=4326).set_index('idx')
gdf_riv0.to_file(join(mdir, 'gis', 'nextxy_outlets.geojson'), driver='GeoJSON')#.explore('rivwth', vmin=min_rivwth, vmax=500)
gdf_riv0
